<a href="https://colab.research.google.com/github/martin-fabbri/colab-notebooks/blob/master/deeplearning.ai/tf/tf_name_entity_recognition_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Download Kaggle Dataset
#@markdown Dataset: Annotated Corpus for Named Entity Recognition <br>
#@markdown https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus
#@markdown ___

kaggle_dataset_id = "abhinavwalia95/entity-annotated-corpus" #@param {type:"string"}

!pip install -q kaggle
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir -p ~/.kaggle
!cp /content/gdrive/My\ Drive/kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d {kaggle_dataset_id}
!ls -l /content
!unzip -o /content/entity-annotated-corpus

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
entity-annotated-corpus.zip: Skipping, found more recently modified local copy (use --force to force download)
total 195268
-rw-r--r-- 1 root root  27703149 Dec 24 07:44 entity-annotated-corpus.zip
drwx------ 5 root root      4096 Dec 24 07:44 gdrive
-rw-r--r-- 1 root root 157030359 Sep 20  2019 ner.csv
-rw-r--r-- 1 root root  15208151 Sep 20  2019 ner_dataset.csv
drwxr-xr-x 1 root root      4096 Dec 21 17:29 sample_data
Archive:  /content/entity-annotated-corpus.zip
  inflating: ner.csv                 
  inflating: ner_dataset.csv         


In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-d60ce961-179e-66ef-3c51-3b5e2c11c2e4)


In [32]:
import math
import pathlib
import shutil
import tempfile

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tf_ad
from numpy.random import seed
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import (
    LSTM,
    Bidirectional,
    Dense,
    Embedding,
    TimeDistributed,
)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.random import set_seed

set_seed(42)
seed(42)

logdir = pathlib.Path(tempfile.mkdtemp())/"tensorflow_logs"
shutil.rmtree(logdir, ignore_errors=True)

ImportError: ignored

## Load the dataset

In [4]:
df = pd.read_csv("ner.csv", encoding="ISO-8859-1", error_bad_lines=False)
df.head()

b'Skipping line 281837: expected 25 fields, saw 34\n'


,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [5]:
data = df[["sentence_idx", "word", "tag"]]
data.head(15)

,sentence_idx,word,tag
0,1.0,Thousands,O
1,1.0,of,O
2,1.0,demonstrators,O
3,1.0,have,O
4,1.0,marched,O
5,1.0,through,O
6,1.0,London,B-geo
7,1.0,to,O
8,1.0,protest,O
9,1.0,the,O


In [6]:
data["tag"].value_counts()

O        889973
B-geo     37525
B-tim     20193
B-org     20184
I-per     17382
B-per     17011
I-org     16537
B-gpe     16392
I-geo      7409
I-tim      6298
B-art       434
B-eve       348
I-eve       297
I-art       280
I-gpe       229
B-nat       226
I-nat        76
Name: tag, dtype: int64

In [7]:
agg_func = lambda s: [(w, t) for w, t in zip(s["word"], s["tag"])]
data.groupby("sentence_idx").apply(agg_func)

sentence_idx
1.0        [(Thousands, O), (of, O), (demonstrators, O), ...
2.0        [(Families, O), (of, O), (soldiers, O), (kille...
3.0        [(They, O), (marched, O), (from, O), (the, O),...
4.0        [(Police, O), (put, O), (the, O), (number, O),...
5.0        [(The, O), (protest, O), (comes, O), (on, O), ...
                                 ...                        
47955.0    [(Indian, B-gpe), (border, O), (security, O), ...
47956.0    [(Indian, B-gpe), (officials, O), (said, O), (...
47957.0    [(Two, O), (more, O), (landed, O), (in, O), (f...
47958.0    [(They, O), (say, O), (not, O), (all, O), (of,...
47959.0    [(Indian, B-gpe), (forces, O), (said, O), (the...
Length: 35177, dtype: object

In [8]:
class SentenceGetter():
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"], s["tag"])]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped[f"Sentence: {self.n_sent}"]
            self.n_sent += 1
            return s
        except:
            return None

getter = SentenceGetter(data)
sentences = getter.sentences
print(repr(sentences[0][:4]), "...")

[('Thousands', 'O'), ('of', 'O'), ('demonstrators', 'O'), ('have', 'O')] ...


In [21]:
tokens = data["word"].unique()
tokens = {"unk" if t is math.nan or isinstance(t, float) else t for t in tokens}
num_tokens = len(tokens)

maxlen = max([len(t) for t in tokens])

tags = data["tag"].unique()
tags = {"unk" if t is math.nan or isinstance(t, float) else t for t in tags}
num_tags = len(tags)

print(f"Num tokenks: {num_tokens:,}  Num tags: {num_tags}  maxlen: {maxlen}")

Num tokenks: 30,173  Num tags: 18  maxlen: 64


In [28]:
token2idx = {token: idx for idx, token in enumerate(tokens)}
idx2token = {idx: token for idx, token in enumerate(tokens)}
tag2idx = {tag: idx for idx, tag in enumerate(tags)}
idx2tag = {idx: tag for idx, tag in enumerate(tags)}

unk_token_idx = token2idx['unk']
unk_tag_idx = tag2idx['unk']

In [31]:
X = [[token2idx[t] for t, _ in s] for s in sentences]
X = pad_sequences(X, maxlen=maxlen, padding="post", value=unk_token_idx)

y = [[tag2idx[t] for _, t in s] for s in sentences]
y = pad_sequences(y, maxlen=maxlen, padding="post", value=unk_tag_idx)

In [36]:
!pip install tf2crf

In [50]:
from tensorflow_addons.text.crf import 
word_embedding_size = 150

model = Sequential([
    Embedding(input_dim=num_tokens, output_dim=word_embedding_size),
    Bidirectional(LSTM(
        units=word_embedding_size,
        return_sequences=True,
        dropout=0.5,
        recurrent_dropout=0.5
    )),
    LSTM(
        units=word_embedding_size * 2,
        return_sequences=True,
        recurrent_dropout=0.5
    ),
    crf_log_likelihood(TimeDistributed(Dense(num_tags, activation="relu"))),
])

TypeError: ignored

AttributeError: ignored